<a href="https://colab.research.google.com/github/talao018/FSL-Recognition_JMP/blob/main/FSL_JMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Clone YOLOv5 and install requirements
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17493, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17493 (delta 1), reused 0 (delta 0), pack-reused 17490 (from 3)
Receiving objects: 100% (17493/17493), 16.56 MiB | 17.02 MiB/s, done.
Resolving deltas: 100% (11996/11996), done.
/content/yolov5
ERROR: Operation cancelled by user


In [ ]:
# STEP 2: Disable Weights & Biases logging (optional)
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
# STEP 3: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# STEP 4: Copy the dataset ZIP from Drive and unzip it
import os
import zipfile

gdrive_zip_path = '/content/drive/MyDrive/fsl_database/fsl_database.zip'  # Fixed path
dataset_dir = '/content/fsl_dataset'  # Output location in Colab

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

with zipfile.ZipFile(gdrive_zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/fsl_database/fsl_database.zip'

In [ ]:
# STEP 5: Rename .data to data.yaml (YOLOv5 requires .yaml)
original_yaml = os.path.join(dataset_dir, "data")
if os.path.exists(original_yaml):
    os.rename(original_yaml, os.path.join(dataset_dir, "data.yaml"))

In [ ]:
# STEP 6: Fix YAML paths (make them absolute)
yaml_path = os.path.join(dataset_dir, "data.yaml")

fixed_yaml = """train: /content/fsl_dataset/train/images
val: /content/fsl_dataset/valid/images
test: /content/fsl_dataset/test/images

nc: 5
names: ['Mahal Kita', 'Naintindihan', 'Paumanhin', 'Salamat', 'Walang Anuman']
"""

with open(yaml_path, 'w') as f:
    f.write(fixed_yaml)

In [ ]:
# STEP 7: Verify folder structure
!ls /content/fsl_dataset

In [ ]:
%cd /content/yolov5

In [ ]:
# STEP 8: Train YOLOv5 model
!python train.py --img 640 --batch 16 --epochs 50 --data /content/fsl_dataset/data.yaml --weights yolov5s.pt --cache

In [ ]:
# Step 9–11: Capture → Detect → Display
from IPython.display import display, Javascript, Image as IPImage
from google.colab.output import eval_js
from base64 import b64decode
import os

# Take photo from webcam
def take_photo(filename='webcam.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = '📸 Take Photo';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();

            const data = canvas.toDataURL('image/jpeg', quality);
            div.remove();
            return data;
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

# 1. Take photo
image_path = take_photo('webcam.jpg')

# 2. Delete old detection results
!rm -rf /content/yolov5/runs/detect

# 3. Run detection using correct detect.py path
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source webcam.jpg

# 4. Display detection result
detect_dir = "/content/yolov5/runs/detect"
if os.path.exists(detect_dir):
    subdirs = sorted([d for d in os.listdir(detect_dir) if not d.startswith('.')])
    latest = os.path.join(detect_dir, subdirs[-1])
    img_path = os.path.join(latest, "webcam.jpg")
    if os.path.exists(img_path):
        display(IPImage(img_path))
    else:
        print("⚠️ Detection image not found.")
else:
    print("⚠️ Detection folder not created. Check if detect.py ran successfully.")